In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("../../datasets/dataset_removed_repeated_charcter.csv")
df.head()

,comment_text,span,label
0,Great post !\r\nJustin at the 150 apologizing...,[],0
1,Mark Hume has always been the stereotypical Va...,"[149, 150, 151, 152, 153, 154, 155, 156, 157, ...",1
2,"Freakin maggots, I swear.",[],1
3,so far the stupidest comments are about the NR...,[],1
4,‘Stupidity is a more dangerous enemy of the go...,[],0


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X=df["comment_text"]
Y=df["label"]
print(X)
print(Y)

0         Great post !\r\nJustin at the 150 apologizing...
1        Mark Hume has always been the stereotypical Va...
2                                Freakin maggots, I swear.
3        so far the stupidest comments are about the NR...
4        ‘Stupidity is a more dangerous enemy of the go...
                               ...                        
20466    He might have been sleazy but trump is both sl...
20467    Seems like the single payer system would be be...
20468                                               Idiot!
20469    My only Question is:  3 staff attorneys chosen...
20470    "They look like fools", ignorance must be blis...
Name: comment_text, Length: 20471, dtype: object
0        0
1        1
2        1
3        1
4        0
        ..
20466    0
20467    0
20468    1
20469    0
20470    1
Name: label, Length: 20471, dtype: int64


In [5]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [7]:
from transformers import BertTokenizer,AutoModel

In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
MAX_LEN=512

In [9]:
output=tokenizer.encode_plus("Hello all",max_length=MAX_LEN,padding='max_length',
                return_token_type_ids=True,
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt')
output
output['input_ids'].flatten().shape

torch.Size([512])

In [ ]:
output=tokenizer("Hello all",max_length=MAX_LEN,padding='max_length')
output

In [20]:
import torch
import torch.nn as nn

class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, df, tokenizer, max_len):
            self.df = df
            self.tokenizer = tokenizer
            self.max_len= max_len
            self.title=X
            self.targets=Y
        def __len__(self):
            return len(self.title)

        def __getitem__(self,index):
            title = str(self.title[index])
            title = " ".join(title.split())

            inputs = self.tokenizer(
                title,None,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                return_token_type_ids=True,
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )

            # print(f"[+]Index ok {index}",inputs['input_ids'].flatten().shape)
            # print(f"[^]Have text {title}")
            # return {
            #     'input_ids': inputs['input_ids'].flatten(),
            #     'attention_mask': inputs['attention_mask'].flatten(),
            #     'token_type_ids': inputs['token_type_ids'].flatten(),
            #     'targets': torch.FloatTensor(self.targets[index])
            # }
            target=self.targets[index]
            return inputs,target
                

In [21]:
train_size=0.8
train_df={}
train_df["message"]=X_train
train_df["label"]=Y_train

train_df=pd.DataFrame(train_df)


val_df={}

val_df["message"]=X_test
val_df["label"]=Y_test
val_df=pd.DataFrame(val_df)

print(train_df.head())
print(val_df.head())

                                                 message  label
56     These bums ripped off the US for many tens of ...      1
15707  Why is the G&M giving KOL air-time?? \r\nThis ...      1
808    Putin must be wetting his pants with excitemen...      0
18810  These idiots disgust me! Just because they sho...      1
4637   You would think. But then you are looking at a...      0
                                                 message  label
16208                                   We elected them.      0
8169   Giving a star to your own comment. How patheti...      1
4961   And there we go. Another attack on Canadians' ...      1
8428   ". the angry populist movement that lashes out...      1
2505   Please move to Canada since you only care abou...      1


In [22]:
train_dataset=CustomDataset(train_df,tokenizer,MAX_LEN)
val_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)


In [23]:
train_data_loader=torch.utils.data.DataLoader(
    train_dataset,
    # shuffle=True,
    batch_size=64,
    num_workers=0

)
val_data_loader=torch.utils.data.DataLoader(
    val_dataset,
    # shuffle=True,
    batch_size=64,
    num_workers=0

)

In [24]:
device= torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [37]:
class ToxicClassiferModel(nn.Module):

    def __init__(self):
        super().__init__()

        self.BERTLayer=AutoModel.from_pretrained("bert-base-uncased",return_dict=True)
        self.dropoutLayer=nn.Dropout(0.3)
        self.fc=nn.Linear(768,1)
    
    def forward(self,input_ids,attention_mask,token_type_id):
        print("[+]Forward")
        output=self.BERTLayer(input_ids,attention_mask,token_type_id,return_dict=True)
        output=self.dropoutLayer(output.pooler_output)
        output=self.fc(output)

        return output

model=ToxicClassiferModel()
model.to(device)


ToxicClassiferModel(
  (BERTLayer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [38]:
def loss_fn(outputs,target):
    return nn.BCEWithLogitsLoss()(outputs,target)

optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001)

In [ ]:
epochs=10

for epoch in range(epochs):
    val_loss=0
    model.train()

    for index,(inputs,target) in enumerate(train_data_loader):
    
        # print(f'[+]Batch {index}')
        input_ids=inputs['input_ids'].view(64,512).to(device)
        attention_mask=inputs['attention_mask'].view(64,512).to(device)
        token_type_ids=inputs['token_type_ids'].view(64,512).to(device)
        print(input_ids.shape)
        print(attention_mask.shape)
        print(token_type_ids.shape)
        target=target.to(device)
        output=model(input_ids,attention_mask,token_type_ids)
        print("[+]Output",output[0])
        break
    print(f'[+]Epoch {epoch}')
  